In [32]:
import numpy as np
from sklearn import preprocessing as pp
from sklearn import datasets as ds 
from sklearn.neighbors import KNeighborsClassifier as KNCfrom sklearn.model_selection import StratifiedKFold


In [73]:
df = ds.load_breast_cancer()
X,y = df.data, df.target

#Escalando para probar el efecto de escalar
X[:,1] = 10*X[:,1]
X[:,3] = 1000*X[:,1]
X[-1,:] = 1000*X[-1,:]

skf = StratifiedKFold(n_splits=2)
fold=0
n=5

for idx_train, idx_test in skf.split(X,y):
    fold +=1
    X_train, X_test = X[idx_train], X[idx_test]
    y_train, y_test = y[idx_train], y[idx_test]
    
    modelo_original = KNC(n_neighbors=n)
    modelo_maxmin = KNC(n_neighbors=n)
    modelo_standar = KNC(n_neighbors=n)
    modelo_robust = KNC(n_neighbors=n)
    
    maxmin_normalization = pp.MinMaxScaler().fit(X_train)
    z_normalization = pp.StandardScaler().fit(X_train)
    robust_normalization = pp.RobustScaler().fit(X_train)
    
    
    X_train_maxmin = maxmin_normalization.transform(X_train)
    X_test_maxmin = maxmin_normalization.transform(X_test)
    
    X_train_stdsc = z_normalization.transform(X_train)
    X_test_stdsc = z_normalization.transform(X_test)
    
    X_train_robust = robust_normalization.transform(X_train)
    X_test_robust = robust_normalization.transform(X_test)

    
    modelo_original.fit(X_train,y_train)
    modelo_maxmin.fit(X_train_maxmin,y_train)
    modelo_standar.fit(X_train_stdsc,y_train)
    modelo_robust.fit(X_train_robust,y_train)
    
    score_original = modelo_original.score(X_test,y_test)
    score_maxmin = modelo_maxmin.score(X_test_maxmin,y_test)
    score_standar = modelo_standar.score(X_test_stdsc,y_test)
    score_robust = modelo_robust.score(X_test_robust,y_test)
    
    print(f"***Fold {fold}***")
    print("Train")
    print(f"Performance original {modelo_original.score(X_train,y_train)}")
    print(f"Performance max_min {modelo_maxmin.score(X_train,y_train)}")
    print(f"Performance z-score {modelo_standar.score(X_train,y_train)}")
    print(f"Performance robust {modelo_robust.score(X_train,y_train)}")
    print("Test")
    print(f"Performance original {score_original}")
    print(f"Performance max_min {score_maxmin}")
    print(f"Performance z-score {score_standar}")
    print(f"Performance robust {score_robust}")

***Fold 1***
Train
Performance original 0.8063380281690141
Performance max_min 0.3732394366197183
Performance z-score 0.3732394366197183
Performance robust 0.3732394366197183
Test
Performance original 0.7333333333333333
Performance max_min 0.9157894736842105
Performance z-score 0.9052631578947369
Performance robust 0.9578947368421052
***Fold 2***
Train
Performance original 0.8070175438596491
Performance max_min 0.6280701754385964
Performance z-score 0.6280701754385964
Performance robust 0.6280701754385964
Test
Performance original 0.6690140845070423
Performance max_min 0.9683098591549296
Performance z-score 0.9647887323943662
Performance robust 0.9683098591549296


In [102]:
def quantile_normalization(X):
    X_copy = np.copy(X)
    indexes_list=[]
    for feature in range(X.shape[1]):
        indexes = np.argsort(X[:,feature])
        sorted_ = np.sort(X[:,feature])
        X_copy[:,feature] = sorted_
        indexes_list.append(indexes)
    means = np.mean(X_copy,axis=1)
    X_copy = np.multiply(means[:,None],np.ones(X_copy.shape))
    
    for feature in range(X.shape[1]):
        indexes = indexes_list[0]
        X_copy[indexes,feature] = X_copy[:,feature]
    
    return(X_copy)
quantile_normalization(X_train)

array([[5477.9835854 , 5477.9835854 , 5477.9835854 , ..., 5477.9835854 ,
        5477.9835854 , 5477.9835854 ],
       [5087.9304936 , 5087.9304936 , 5087.9304936 , ..., 5087.9304936 ,
        5087.9304936 , 5087.9304936 ],
       [5980.96540427, 5980.96540427, 5980.96540427, ..., 5980.96540427,
        5980.96540427, 5980.96540427],
       ...,
       [5498.0879687 , 5498.0879687 , 5498.0879687 , ..., 5498.0879687 ,
        5498.0879687 , 5498.0879687 ],
       [5994.36610347, 5994.36610347, 5994.36610347, ..., 5994.36610347,
        5994.36610347, 5994.36610347],
       [4947.0419488 , 4947.0419488 , 4947.0419488 , ..., 4947.0419488 ,
        4947.0419488 , 4947.0419488 ]])